# Test BED data structures

The test cases this structure should fullfill so far are
1. simple one scalar measurment per design vector point (e.g: travel time source location)
2. get combinations of arrival times for a number of design points  (e.g: travel time tomography)
3. Deal with waveforms, meaning a single design point can give a measurment vector (e.g.: full waveform inversion)
4. Deal with combined measurments of waveforms and scalar measurments
5. 

the best is probably using a user defined function.

In [1]:
import numpy as np
import torch
import h5py

from  geobed.design2data_helpers import *

In [2]:
# set up general parameters
n_prior = 100

n_design_points = 5

In [3]:
def simple_sequential_design(design_budget, design_meta_data, design2data):
    budget = 0
    optimal_design = []

    while budget <= design_budget:
        
        EIG_optimal = -np.inf
        # loop through all the designs 
        for name, meta_data in design_meta_data.items():
            
            # if the budget is not exceeded, add the design to the design set
            if design_budget < budget + meta_data['cost']:
                continue
            temp_design = optimal_design + [name]

            data = design2data(temp_design, design_meta_data, prior_samples=torch.ones((n_prior, 1)))

            EIG = torch.sum(torch.sum(torch.log(data) * data, dim=-1), dim=0)
                                                        
            if EIG > EIG_optimal:
                EIG_optimal = EIG
                candidate = name
        
        if EIG_optimal == -np.inf:
            break
        
        budget += design_meta_data[candidate]['cost']
        optimal_design.append(candidate)            
                
    return optimal_design

## Test Case (1)

### 1.1 Writing Data

In [4]:
design_names_case_1   = [str(i) for i in range(n_design_points)]

design_meta_data_1 = {
    '1': {'file': 'data/case1_data.hdf5', 'dataset': 'data', 'index': 0, 'cost': 1, 'x': 1, 'y': 2},
    '2': {'file': 'data/case1_data.hdf5', 'dataset': 'data', 'index': 1, 'cost': 1, 'x': 2, 'y': 3},
    '3': {'file': 'data/case1_data.hdf5', 'dataset': 'data', 'index': 2, 'cost': 1, 'x': 3, 'y': 4},
    '4': {'file': 'data/case1_data.hdf5', 'dataset': 'data', 'index': 3, 'cost': 1, 'x': 4, 'y': 5},
    '5': {'file': 'data/case1_data.hdf5', 'dataset': 'data', 'index': 4, 'cost': 1, 'x': 5, 'y': 6},
}

with h5py.File("case1_data.hdf5", "w") as f:
    
    np.random.seed(0)
    data = f.create_dataset("data", (n_prior, n_design_points, 1))        
    data[:] = np.random.uniform(1, 2, (n_prior, n_design_points, 1))

### 1.2 Constructing designs

In [5]:
def lookup_1to1_design(temp_name_list, design_meta_data, prior_samples):
    
    temp_name_list = list(set(temp_name_list)) # remove duplicates 
    design_meta_list = [design_meta_data[n] for n in temp_name_list]
    
    n_prior = prior_samples.shape[0]
    data = torch.zeros((n_prior, len(temp_name_list), 1))    
    
    for i, design_meta in enumerate(design_meta_list):
        with h5py.File(design_meta['file'], "r") as df:
            data[:, i, :] = torch.from_numpy(df[design_meta['dataset']][:, design_meta['index'], :])

    return data.flatten(start_dim=-2)

In [6]:
optimal_design_1 = simple_sequential_design(3, design_meta_data_1, lookup_1to1_design)

print(optimal_design_1)

['4', '2', '5']


## Test Case (2)

In [7]:
design_names_2 = [str(i) for i in range(n_design_points)]

design_meta_data_2 = {
    '1': {'file': 'data/case2_data.hdf5', 'dataset': 'data', 'cost': 1, 'x': 1, 'y': 2},
    '2': {'file': 'data/case2_data.hdf5', 'dataset': 'data', 'cost': 1, 'x': 2, 'y': 3},
    '3': {'file': 'data/case2_data.hdf5', 'dataset': 'data', 'cost': 1, 'x': 3, 'y': 4},
    '4': {'file': 'data/case2_data.hdf5', 'dataset': 'data', 'cost': 1, 'x': 4, 'y': 5},
    '5': {'file': 'data/case2_data.hdf5', 'dataset': 'data', 'cost': 1, 'x': 4, 'y': 5},
    }

with h5py.File("case2_data.hdf5", "w") as f:

    np.random.seed(0)
    data = f.create_dataset("data", (n_prior, n_design_points * (n_design_points-1) // 2, 1))        
    data[:] = np.random.uniform(1, 2, (n_prior, n_design_points * (n_design_points-1) // 2, 1))


In [8]:
def lookup_interstation_design(temp_name_list, design_meta_data, prior_samples):

    temp_name_list = list(set(temp_name_list)) # remove duplicates 
    design_meta_list = [design_meta_data[n] for n in temp_name_list]

    if len(design_meta_list) == 1:
        indices = []
    else:
        int_names = torch.tensor([int(name_i)-1 for name_i in temp_name_list])        
        
        indices = (torch.combinations(int_names)).tolist()    
        indices = [list(sorted(i)) for i in indices]
        
        all_indices = zip(*torch.tril_indices(n_design_points, n_design_points, offset=-1).tolist())
        all_indices = [list(sorted(i)) for i in all_indices]
        
        indices = [i for i, ind in enumerate(all_indices) if ind in indices]
    
    n_prior = prior_samples.shape[0]
    data = torch.zeros((n_prior, len(indices), 1))    
    
    filename = design_meta_list[0]['file']
    dataset_name = design_meta_list[0]['dataset']
        
    with h5py.File(filename, "r") as df:
        data = torch.from_numpy(df[dataset_name][:, indices, :])
        
    return data.flatten(start_dim=-2)

In [9]:
optimal_design_2 = simple_sequential_design(3, design_meta_data_2, lookup_interstation_design)

print(optimal_design_2)

['1', '5', '3']


## Test Case (3)

In [18]:
n_waveform = 60

design_names_3   = [str(i) for i in range(n_design_points)]

design_meta_data_3 = {
    '1': {'file': 'data/case3_data.hdf5', 'dataset': 'data', 'index': 0, 'cost': 1, 'x': 1, 'y': 2},
    '2': {'file': 'data/case3_data.hdf5', 'dataset': 'data', 'index': 1, 'cost': 1, 'x': 2, 'y': 3},
    '3': {'file': 'data/case3_data.hdf5', 'dataset': 'data', 'index': 2, 'cost': 1, 'x': 3, 'y': 4},
    '4': {'file': 'data/case3_data.hdf5', 'dataset': 'data', 'index': 3, 'cost': 1, 'x': 4, 'y': 5},
    '5': {'file': 'data/case3_data.hdf5', 'dataset': 'data', 'index': 4, 'cost': 1, 'x': 5, 'y': 6},
}

with h5py.File("case3_data.hdf5", "w") as f:
    data = f.create_dataset("data", (n_prior, n_design_points, n_waveform))        
    data[:] = np.random.uniform(1, 2, (n_prior, n_design_points, n_waveform))

In [20]:
optimal_design_3 = simple_sequential_design(3, design_meta_data_3, lookup_1to1_design_variable_length)

print(optimal_design_3)

['1', '4', '3']


## Test Case (4)

In [12]:
n_waveform_list = [10, 20, 30, 40, 50]

design_names_4   = [str(i) for i in range(n_design_points)]

design_meta_data_4 = {
    '1': {'file': 'data/case4_data.hdf5', 'dataset': 'data', 'index': 0, 'cost': 1, 'x': 1, 'y': 2},
    '2': {'file': 'data/case4_data.hdf5', 'dataset': 'data', 'index': 1, 'cost': 1, 'x': 2, 'y': 3},
    '3': {'file': 'data/case4_data.hdf5', 'dataset': 'data', 'index': 2, 'cost': 1, 'x': 3, 'y': 4},
    '4': {'file': 'data/case4_data.hdf5', 'dataset': 'data', 'index': 3, 'cost': 1, 'x': 4, 'y': 5},
    '5': {'file': 'data/case4_data.hdf5', 'dataset': 'data', 'index': 4, 'cost': 1, 'x': 5, 'y': 6},
}

with h5py.File("case4_data.hdf5", "w") as f:
    
    variable_dt = h5py.vlen_dtype(np.dtype('float64'))
    data = f.create_dataset('data', (n_prior, n_design_points,), dtype=variable_dt)
    
    for i_station, n_waveform in enumerate(n_waveform_list):
        data[:, i_station] = np.random.uniform(1, 2, (n_prior, n_waveform))

In [13]:
def lookup_1to1_design_variable_length(temp_name_list, design_meta_data, prior_samples):
    
    temp_name_list = list(set(temp_name_list)) # remove duplicates 
    design_meta_list = [design_meta_data[n] for n in temp_name_list]
                
    data = []
    
    for i, design_meta in enumerate(design_meta_list):            
        
        with h5py.File(design_meta['file'], "r") as df:
            i_data = np.stack(df[design_meta['dataset']][:, design_meta['index']])

            data.append(i_data)
                                      
    return torch.from_numpy(np.concatenate(data, axis=-1))
                                    
     

In [14]:
optimal_design_4 = simple_sequential_design(3, design_meta_data_4, lookup_1to1_design_variable_length)

print(optimal_design_4)

['5', '4', '3']


## Test Case (5)

In [15]:
n_waveform_list = [10, 20, 30, 40, 50]

design_names   = [str(i) for i in range(n_design_points)]

design_meta_data = {
    '1': {'cost': 1, 'x': 1, 'y': 2, 'forward_function': torch.randint},
    '2': {'cost': 1, 'x': 2, 'y': 3, 'forward_function': torch.randint},
    '3': {'cost': 1, 'x': 3, 'y': 4, 'forward_function': torch.randint},
    '4': {'cost': 1, 'x': 4, 'y': 5, 'forward_function': torch.randint},
    '5': {'cost': 1, 'x': 5, 'y': 6, 'forward_function': torch.randint},
}

In [16]:
def constructor_1to1_design(temp_name_list, design_meta_data, prior_samples):
    
    temp_name_list = list(set(temp_name_list)) # remove duplicates 
    design_meta_list = [design_meta_data[n] for n in temp_name_list]
    
    n_prior = prior_samples.shape[0]
    
    data = torch.zeros((n_prior, len(temp_name_list), 10))
        
    for i, d_meta in enumerate(design_meta_list):
        data[:, i, :] = d_meta['forward_function'](1, 6, (n_prior, 10))

    return data.flatten(start_dim=-2)

In [17]:
design_budget = 3

# simple sequential design test case

budget = 0
optimal_design = []

while budget <= design_budget:
    
    EIG_optimal = -np.inf
    # loop through all the designs 
    for name, meta_data in design_meta_data.items():
        
        # if the budget is not exceeded, add the design to the design set
        if design_budget < budget + meta_data['cost']:
            continue
        temp_design = optimal_design + [name]

        data = constructor_1to1_design(temp_design, design_meta_data, prior_samples=torch.ones((n_prior, 1)))
                    
        EIG = torch.sum(torch.sum(torch.log(data) * data, dim=-1), dim=0)
                                                    
        if EIG > EIG_optimal:
            EIG_optimal = EIG
            candidate = name
    
    if EIG_optimal == -np.inf:
        break
    
    budget += design_meta_data[candidate]['cost']
    optimal_design.append(candidate)            
            
print(optimal_design)

['5', '1', '4']
